In [1]:
import json
import os
import sys
from datetime import datetime
import pandas as pd
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, EarlyStoppingCallback
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

MODEL_NAME = 'distilbert-base-uncased'
MAX_LENGTH = 512

FILENAME_TEST = 'test.csv'
DIR_OUTPUT = 'results'

def get_ts():
    return datetime.utcnow().replace(microsecond=0).isoformat()
# end


class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)
    # end
# end

def read_passages(path_data, path_label, test_size=0):
    
    df = pd.read_csv(path_data)

    documents = df['processed'].to_list()
    labels_str = df['target'].to_list()

    samples = documents
    
    with open(path_label, 'r') as file:
        labels_list = sorted(json.load(file))
    # end
    
    labels_all = {l: idx for idx, l in enumerate(labels_list)}
    
    labels = [labels_all[label_str] for label_str in labels_str]

    if test_size > 0:
        return train_test_split(samples, labels, test_size=test_size, stratify=labels, random_state=234), labels_list
    else:
        return (samples, samples, labels, labels), labels_list
    # end
# end


def compute_metrics(pred):
    labels = pred.label_ids.reshape(-1)
    preds = pred.predictions.argmax(-1).reshape(-1)

    accuracy = accuracy_score(y_true=labels, y_pred=preds)
    precision = precision_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    recall = recall_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    f1 = f1_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}
# end

def main_train_and_evaluate(name_train, path_train, path_label, path_test, path_output):
    print('[{}] start main_train_and_evaluate with {} {}'.format(get_ts(), path_train, path_test))

    model_name = MODEL_NAME
    max_length = MAX_LENGTH
    output_dir = DIR_OUTPUT
    
    (train_samples, valid_samples, train_labels, valid_labels), target_names = read_passages(path_train, path_label, 0.1)

    tokenizer = DistilBertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
    train_encodings = tokenizer.batch_encode_plus(train_samples, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    valid_encodings = tokenizer.batch_encode_plus(valid_samples, truncation=True, padding=True, max_length=max_length, return_tensors='pt')

    train_dataset = SimpleDataset(train_encodings, train_labels)
    valid_dataset = SimpleDataset(valid_encodings, valid_labels)

    model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))

    training_args = TrainingArguments(
        output_dir=output_dir,  # output directory
        num_train_epochs=10,  # total number of training epochs
        per_device_train_batch_size=2,  # batch size per device during training
        per_device_eval_batch_size=2,  # batch size for evaluation
        warmup_steps=0,  # number of warmup steps for learning rate scheduler
        weight_decay=0.01,  # strength of weight decay
        logging_dir='./logs',  # directory for storing logs
        load_best_model_at_end=True,
        # load the best model when finished training (default metric is loss)    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
        logging_steps=1,  # log & save weights each logging_steps
        evaluation_strategy="epoch",  # evaluate each `logging_steps`
        learning_rate=2e-5,
        save_strategy='epoch',
        save_total_limit=5,
        metric_for_best_model='f1'
    )

    trainer = Trainer(
        model=model,  # the instantiated Transformers model to be trained
        args=training_args,  # training arguments, defined above
        train_dataset=train_dataset,  # training dataset
        eval_dataset=valid_dataset,  # evaluation dataset
        compute_metrics=compute_metrics,  # the callback that computes metrics of interest
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )
    
    # trainer = Trainer(
    #     model=model,  # the instantiated Transformers model to be trained
    #     args=training_args,  # training arguments, defined above
    #     train_dataset=train_dataset,  # training dataset
    #     eval_dataset=valid_dataset,  # evaluation dataset
    #     compute_metrics=compute_metrics
    # )

    print('[{}] start training...'.format(get_ts()))
    trainer.train()

    info_state_model = trainer.evaluate()
    print('[{}] finish training.'.format(get_ts()))

    ################## start to do eval ##################

    
    (samples_test, _, indexs_label_test, _), target_names = read_passages(path_test, path_label, 0)
    labels_test = [target_names[index_label_test] for index_label_test in indexs_label_test]

    list_conf_output = []
    list_label_output = []

    for sample_test, label_origin in zip(samples_test, labels_test):
        input_tokenized = tokenizer.encode_plus(sample_test, padding=True, truncation=True, max_length=max_length,
                                                return_tensors='pt').to('cuda')
        with torch.no_grad():
            out = model(**input_tokenized, output_hidden_states=True, output_attentions=True)
        # end

        probas_evaluate = torch.nn.functional.softmax(out.logits, dim=-1)
        answer_evaluate = int(probas_evaluate.argmax().cpu())

        label_evaluate = target_names[answer_evaluate]

        list_conf_output.append(probas_evaluate.cpu().numpy().tolist()[0][answer_evaluate])
        list_label_output.append(label_evaluate)
    # end

    print('[{}] finish testing.'.format(get_ts()))


    pairs_label_conf = [[a,b] for a,b in zip(list_label_output, list_conf_output)]

    filename_output = f'output-{name_train}.json'
    path_file_output = os.path.join(path_output, filename_output)
    
    with open(path_file_output, 'w+') as file:
        file.write(json.dumps(pairs_label_conf))
    # end

    print('[{}] main_train_and_evaluate finished.'.format(get_ts()))
# end

In [ ]:
path_folder_train = 'data/training'
path_test = 'data/test_from_train.csv'
path_label = 'data/labels.json'
path_output = 'data/output_noseed_train'

import numpy as np
import random
import torch

seed_val = 234
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

import os
import subprocess

filenames = [filename for filename in os.listdir(path_folder_train) if filename[0] != '.']

for filename in filenames:
    path_train = os.path.join(path_folder_train, filename)
    name_train = filename.split('.')[0]
    
    main_train_and_evaluate(name_train, path_train, path_label, path_test, path_output)
    
    subprocess.run("rm -rf results", shell=True)
    # subprocess.run("rm -rf mlruns", shell=True)
# end

[2023-04-25T06:58:13] start main_train_and_evaluate with data/training/202210130649.csv data/test_from_train.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

[2023-04-25T06:58:24] start training...


/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-1-95d5a3d64ccb>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.002800,0.286558,0.935714,0.937819,0.935714,0.934840
2,3.193900,0.350307,0.942857,0.948155,0.942857,0.942029
3,0.000200,0.175727,0.971429,0.972420,0.971429,0.971318
4,0.000700,0.114167,0.982143,0.982827,0.982143,0.981985
5,0.000100,0.100485,0.985714,0.985788,0.985714,0.985619
6,0.316800,0.093029,0.989286,0.989713,0.989286,0.989186
7,0.000100,0.092751,0.985714,0.985788,0.985714,0.985619
8,0.000200,0.090059,0.989286,0.989713,0.989286,0.989186
9,0.000100,0.088186,0.989286,0.989547,0.989286,0.989143
10,0.000000,0.091299,0.989286,0.989713,0.989286,0.989186


...

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

[2023-04-25T11:40:21] start training...


<ipython-input-1-95d5a3d64ccb>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.401200,0.350869,0.922078,0.924721,0.922078,0.920969
2,0.001000,0.230174,0.952381,0.951673,0.952381,0.950971
3,0.000300,0.184595,0.969697,0.970288,0.969697,0.968869
4,0.000100,0.216047,0.965368,0.965178,0.965368,0.964572
5,0.000100,0.215860,0.969697,0.969849,0.969697,0.969051
6,0.000200,0.308363,0.961039,0.961843,0.961039,0.960207
7,0.000000,0.290045,0.965368,0.965726,0.965368,0.964480


<ipython-input-1-95d5a3d64ccb>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
<ipython-input-1-95d5a3d64ccb>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
<ipython-input-1-95d5a3d64ccb>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
<ipython-input-1-95d5a3d64ccb>:29: UserWarning: To copy construct from a tensor, it is recommende

In [ ]:
print('hello')